# 2.1 Training models with Anvil
<div style="text-align: center">
<img src="anvil_diagram.png" alt="Anvil diagram" width="500"/>  
</div>

### Background

Anvil is our primary infrastructure for model training and evaluation, built to support scalable, reproducible, and rigorous development of ADMET prediction models. Recognizing that building the best models requires training many variants, ensuring their reproducibility, and enabling robust performance comparisons, Anvil centers around a YAML-based recipe system. These recipes allow users to specify model architectures and training procedures in a standardized, shareable format—minimizing code duplication while supporting both deep learning and traditional machine learning approaches.  

Designed with both internal and external engagement in mind, Anvil aims to lower the barrier for outside users to adopt and fine-tune models by offering simple, transparent workflows. Long-term, it will serve as a foundation for broader community involvement and model reuse.

### Requirements
To run Anvil, you need:
1. A dataset that has been processed with `1.1_Curating_external_datasets.ipynb`.  
2. A `YAML` file with instructions for Anvil. We will show you how to create this file in this notebook.

## 1. Overview

This notebook will walk you through how to run the Anvil model training workflow with human pregnane X receptor (PXR) data processed and cleaned in previous notebooks.

## 2. Creating the YAML file
The heart of an anvil run is in its `YAML` configuration file. Here we specify nearly everything needed to:
- load data
- preprocess it
- split the data appropriately into train/validation/test
- featurize according to model selection
- train the model
- and, finally, validate on the test set (which generates performance metrics and plots)  

We will walkthrough two `YAML` files: one for training a traditional machine learning model (`2.1_anvil_lgbm.yaml`) and one for training a deep learning model (`2.1_anvil_chemprop.yaml`).

## 3. Training a traditional machine learning LightGBM 

Here is a `YAML` file for training a LightGBM (LGBM) model. We are using the previously curated PXR data from ChEMBL. Be sure to read through the comments (in green) to understand each field.  

1. At a minimum, ensure `resource`, `input_col`, and `target_cols` are specified to match your dataset, as these will vary per dataset
2. The `procedure` section may not need much modification, especially if not tweaking parameters, but look it over to make sure it’s sensible

```yaml 
# This spection specifies the input data
data:
  # Specify where anvil will run (inputs, outputs, etc.)
  anvil_dir: demos/anvil_models/lgbm
  # Specify the dataset file
  resource: processed_data/processed_PXR_chembl.parquet
  # TODO: add description of intake
  type: intake
  cat_entry: null
  # Specify input column containing SMILES
  input_col: OPENADMET_CANONICAL_SMILES
  # Specify each (1+) of the target columns, or the column that you're trying to predict
  target_cols:
  - OPENADMET_LOGAC50

# Additional metadata
metadata:
  authors: Cynthia Xu
  biotargets:
  - PXR
  build_number: 0
  description: basic regression using a LightGBM model
  driver: sklearn
  email: cynthia.xu@omsf.io
  name: lgbm_pchembl
  tag: lgbm-PXR-chembl
  tags:
  - openadmet
  - test
  - pchembl
  version: v1

# This section specifies the training procedure
procedure:
  # Featurization specification
  feat:
    # Using a concatenated descriptor and fingerprint as feature vectors
    # See openadmet.models.features
    type: FeatureConcatenator
    # Add parameters for the featurizer. Full description of the featurizer options are in Section 5.
    params:
      featurizers:
        DescriptorFeaturizer:
          descr_type: "desc2d"
        FingerprintFeaturizer:
          fp_type: "ecfp:4"
  
  # This section specifies the model and hyperparameters
  model:
    # Indicate model type, here an LGBM regressor
    # See openadmet.models.architecture for all model types
    type: LGBMRegressorModel
    # Specify model parameters
    params:
      alpha: 0.005
      learning_rate: 0.05

  # Specify data splits
  split:
    # Specify how data will be split
    # See openadmet.models.split
    type: ShuffleSplitter
    # Specify split parameters
    params:
      random_state: 42
      train_size: 0.7
      val_size: 0.0 # For LGBM, no validation set is needed as we are not doing model selection
      test_size: 0.3
    
  # Specify training configuration
  train:
    # Specify the trainer, here GridSearch trainer for a hyperparameter sweep 
    # See openadmet.models.trainer
    type: SKLearnGridSearchTrainer
    # Specify model parameters
    params:
      param_grid:
        n_estimators: [200, 500]
        alpha: [0.005, 0.01]
        learning_rate: [0.05]

# This section specifies the report generation
report:
  # Configure evaluation
  eval:
  # Generate regression metrics
  - type: RegressionMetrics
    params: {}
  # Generate regression plots from cross validation
  - type: SKLearnRepeatedKFoldCrossValidation
    params:
      axes_labels:
      - True pAC50
      - Predicted pAC50
      max_val: 10
      min_val: 3
      pXC50: true
      title: True vs Predicted pAC50 on test set
```

After you have created or modified this `YAML` file to your liking, you can run the workflow with the below command either in a `bash` cell or in your command line:
```
openadmet anvil --recipe-path <your_file.yaml>
```

This may take 5-10 minutes to run, depending on the number of epochs, your hyperparameters (e.g. learning rate), etc.

In [ ]:
%%bash
openadmet anvil --recipe-path 2.1_anvil_lgbm.yaml

The outputs of the Anvil workflow are in `/anvil_training`:  
- `/data` folder includes the split data, saved as `.csv`
- `/recipe_components` folder contains the inputs from the `2.1_anvil_lgbm.yaml` file split by section
- `cross_validation_metrics.json` is the cross validation metrics of the model saved as a `.json` file
- `model.json` is the model's hyperparameters saved as a `.json` file
- `regression_metrics.json` is the regression metrics saved as a `.json` file
- `model.pkl` is the trained model saved as `.pkl` which can be loaded and used for predictions elsewhere
- `cross_validation_regplot.png` is a plot of the cross validation metrics of the model
- `anvil_recipe.yaml` is a copy of the input `.yaml`

Here are the results of above trained LGBM model:

<img src="./anvil_training/cross_validation_regplot.png" alt="LGBM model results" width="1000"/>

## 4. Training a deep learning Chemprop model

Here is a `YAML` file for training OpenADMET's ChemProp model. We are using the same ChEMBL PXR dataset. Be sure to note the different fields required for deep learning.


```yaml
# Section specifying input data
data:
  # Specify where anvil will run (inputs, outputs, etc.)
  anvil_dir: demos/anvil_models/chemprop
  # Specify the dataset file
  resource: processed_data/processed_PXR_chembl.parquet
  # TODO: add description of intake
  type: intake
  # COMMENT ONCE WE KNOW WHAT THIS IS
  cat_entry: null
  # Specify input column containing SMILES
  input_col: OPENADMET_CANONICAL_SMILES
  # Specify each (1+) of the target columns
  target_cols:
  - OPENADMET_LOGAC50

# Additional metadata
metadata:
  authors: Cynthia Xu
  biotargets:
  - PXR
  build_number: 0
  description: basic regression using a ChemProp multitask task model
  driver: pytorch
  email: cynthia.xu@omsf.io
  name: chemprop_pchembl
  tag: chemprop-PXR-chembl
  tags:
  - openadmet
  - test
  - pchembl
  version: v1

# Section specifying training procedure
procedure:
  # Featurization specification
  feat:
    # Using the ChemPropFeaturizer (for ChemProp model)
    # See openadmet.models.features
    type: ChemPropFeaturizer
    # No parameters passed
    params: {}
  
  # Model specification
  model:
    # Indicate model type
    # See openadmet.models.architecture
    type: ChemPropMultiRegressorModel
    # Specify model parameters
    params:
      depth: 3
      ffn_hidden_dim: 1024
      ffn_hidden_num_layers: 1
      message_hidden_dim: 2048
      dropout: 0.2
      batch_norm: True
      messages: bond
      n_tasks: 1

  # Specify data splits
  split:
    # Specify how data will be split
    # See openadmet.models.split
    type: ShuffleSplitter
    # Specify split parameters
    params:
      random_state: 42
      train_size: 0.7
      val_size: 0.2 # For LGBM, set to 0
      test_size: 0.1
    
  # Specify training configuration
  train:
    # Specify the trainer, here LightningTrainer as ChemProp is a PyTorch Lightning model
    # See openadmet.models.trainer
    type: LightningTrainer
    # Specify model parameters
    params:
      accelerator: gpu
      early_stopping: true
      early_stopping_patience: 10
      max_epochs: 20
      monitor_metric: val_loss
      use_wandb: false
      # Don't blindly copy this over!
      wandb_project: demos

# Section specifying report generation
report:
  # Configure evaluation
  eval:
  # Generate regression metrics
  - type: RegressionMetrics
    params: {}
  # Generate regression plots
  - type: RegressionPlots
    params:
      axes_labels:
      - True pChEMBL
      - Predicted pChEMBL
      max_val: 10
      min_val: 3
      pXC50: true
      title: Multitask True vs Predicted pChEMBL on test set
```

This cell will also take around 20-40 minutes to run on a local CPU for a **single epoch**. The model results shown are with a GPU.

In [ ]:
%%bash
openadmet anvil --recipe-path 2.1_anvil_chemprop.yaml

Here are the results of the ChemProp model:

<img src="anvil_training_2025-07-14_86073d/cross_validation_regplot_OPENADMET_LOGAC50.png" alt="ChemProp model results" width="1000"/>

Congrats! You now know how to train models with the Anvil workflow. Explore our [model catalog](https://github.com/OpenADMET/openadmet-models/tree/2f58b521cdf122d8c929f6b64aead96d1378cd6f/openadmet/models) for other model architectures and featurizers

✨✨✨✨✨✨✨